In [1]:
import xarray as xr

from weather_class import WeatherData
from torch.utils.data import DataLoader

# base = xr.open_dataset('C:/Users/23603526/Documents/GitHub/WeatherForecasting/data/era5_training.nc')
base = xr.open_dataset('/teamspace/studios/this_studio/WeatherForecasting/data/era5_testing.nc')
base.load()

<xarray.Dataset> Size: 139MB
Dimensions:         (time: 1464, pressure_level: 1, latitude: 49, longitude: 69)
Coordinates:
  * time            (time) datetime64[ns] 12kB 2020-01-01 ... 2020-12-31T18:0...
  * pressure_level  (pressure_level) float64 8B 850.0
  * latitude        (latitude) float64 392B -35.0 -34.75 -34.5 ... -23.25 -23.0
  * longitude       (longitude) float64 552B 16.0 16.25 16.5 ... 32.5 32.75 33.0
Data variables:
    z               (time, pressure_level, latitude, longitude) float32 20MB ...
    r               (time, pressure_level, latitude, longitude) float32 20MB ...
    t               (time, pressure_level, latitude, longitude) float32 20MB ...
    u               (time, pressure_level, latitude, longitude) float32 20MB ...
    v               (time, pressure_level, latitude, longitude) float32 20MB ...
    w               (time, pressure_level, latitude, longitude) float32 20MB ...
    wind_speed      (time, pressure_level, latitude, longitude) float32 20MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-21T12:48 GRIB to CDM+CF via cfgrib-0.9.1...

In [2]:
window_size = 6
steps = 5
data_split = 'train'
lightning = True


train_loader = DataLoader(WeatherData(base, window_size, steps, data_split, lightning=lightning, only_wspd=False), batch_size=32, shuffle=True)

for i, data in enumerate(train_loader):
    x, F, M, y = data
    break

print(x.shape)
print(F.shape)
print(M.shape)
print(y.shape)

torch.Size([32, 6, 49, 69, 7])
torch.Size([32, 2])
torch.Size([32, 49, 69])
torch.Size([32, 5, 49, 69, 7])
